In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPUs to use

In [2]:
import os
import re
import random
import warnings
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from peft import PeftModel, PeftConfig

from datasets import load_from_disk

from src.metrics import rouge_for_batch, f1_score_at_k_for_batch, jaccard_similarity_for_batch

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Seoul')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:

In [3]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 64)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths and Names

In [5]:
### paths and names

TRAIN_DATA_PATH = 'data/model_dev/model_dev_v4_polyglot_1.3b_train.hf'
EVAL_DATA_PATH = 'data/model_dev/model_dev_v4_polyglot_1.3b_eval.hf'

# MODEL_CHECKPOINT = ''

PEFT_MODEL_ID = '.log/eleutherai_polyglot_ko_1.3b_v4_run_6'

In [6]:
torch_dtype = torch.float16

# Load Model & Tokenizer

In [7]:
# config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [8]:
# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config).to(device).eval()
# tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Load PEFT Model & Tokenizer

In [9]:
peft_config = PeftConfig.from_pretrained(PEFT_MODEL_ID)

In [33]:
# model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, torch_dtype=torch_dtype)

In [11]:
# model = PeftModel.from_pretrained(model, PEFT_MODEL_ID).to(device).eval()
model = PeftModel.from_pretrained(model, PEFT_MODEL_ID).to(device)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Load Data

In [12]:
eval_dataset = load_from_disk(EVAL_DATA_PATH)
print(eval_dataset)

Dataset({
    features: ['id', 'create_date', 'title', 'input_text', 'len_tokenized', 'target_text', 'len_tokenized_target_text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3000
})


In [13]:
rand_idx = random.randint(0, len(eval_dataset)-1)
tokenizer.decode(eval_dataset['input_ids'][rand_idx])
# tokenizer.decode(eval_dataset['labels'][rand_idx])

"generate keyphrases: '딜리버리맨' 신이준, 학폭 가해자로 변신..실감 나는 일진 연기 [스타뉴스 안윤지 기자] /사진=지니TV '딜리버리맨' 방송 캡처 [안윤지 스타뉴스 기자] 배우 신이준이 '딜리버리맨'에서 강렬한 악역 연기로 임팩트를 남겼다. 신이준은 지난 16일 공개된 지니 TV 오리지널 '딜리버리맨'에서 학교 폭력 가해자 최미라 역으로 출연했다. 미라는 동급생 친구들에게 술, 담배 심부름을 시키고 협박을 일삼는 일행의 중심에 있는 인물이다. 사라진 최하율(갈소원 분) 사건에 미라가 얽혀있다는 사실이 밝혀지며 긴장을 놓지 못하게 했다. 이를 표현한 신이준은 실감 나는 일진 연기로 강렬한 인상을 남겼다. 신이준은 최하율의 행방을 묻는 서영민(윤찬영 분) 앞에서는 태연한 표정 연기로 평범한 학생을 표현하고, 뒤에서 동급생들에게 협박을 가할 때는 매섭고 뻔뻔한 표정 연기로 극과 극을 오가며 캐릭터를 실감 나게 구축했다. 또 정확한 딕션과 안정적인 톤으로 극의 몰입도를 높이는 데 기여했다. 아역 배우로 데뷔한 신이준은 드라마 '시그널' '낭만 닥터 김사부' '당신이 잠든 사이에' 등에서 신선한 마스크와 안정적인 연기력으로 주목받아왔다. 또한 '쇼윈도:여왕의 집' '멘탈코치 제갈길'에서 아빠의 불륜을 목격한 딸, 심리적 갈등 겪는 체조선수로 활약하며 임팩트 있는 연기를 보여줬다. 이에 앞으로 새롭게 보여줄 활약을 기대케 한다. 안윤지 기자 zizirong@mtstarnews.com 안윤지 기자 zizirong@mtstarnews.com keyphrases: 딜리버리맨; 신이준; 학폭; 연기; 최미라; 출연; 중심; 최하율; 악역; 연기력"

# Generate

In [68]:
### Beam Search Args
generation_args = {
    'num_beams': 5, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'early_stopping': False,
}

### Diverse Beam Search Args
generation_args = {
    'num_beams': 15, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'num_beam_groups': 3,
    'early_stopping': False,
}

### Sampling Args
generation_args = {
    'do_sample': True, 
    'top_k': 3, 
    'top_p': 0.95, 
}

### Greedy Search Args
generation_args = {
    # 'num_beams': 1, 
}

In [15]:
inputs = eval_dataset[:]

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [ ]:
batch_size = 1
predictions = None
with torch.no_grad(), torch.autocast("cuda"):
    start = 0
    range_end = len(input_ids) if len(input_ids) % batch_size == 0 else len(input_ids) + batch_size
    for idx in tqdm(range(batch_size, range_end, batch_size)):
        ids, mask = input_ids[start:idx], attention_mask[start:idx]
        ids, mask = torch.tensor(ids).to(device), torch.tensor(mask).to(device)

        prediction = model.generate(input_ids=ids, attention_mask=mask, max_length=64, **generation_args)
        if predictions == None:
            predictions = prediction.detach().cpu().tolist()
        else:
            predictions.extend(prediction.detach().cpu().tolist())
        start = idx

predictions_decoded = tokenizer.batch_decode(predictions, skip_special_tokens=True)

In [ ]:
len(predictions), len(inputs['labels'])

# Calculate Metrics

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    prediction_lens = [np.count_nonzero(np.array(pred) != tokenizer.pad_token_id) for pred in predictions]
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE / Mean Generated Length
    print('Calculating ROUGE...')
    result = rouge_for_batch(decoded_labels, decoded_preds)

    # F1@K
    print('Calculating F1@10...')
    result["F1@10"] = f1_score_at_k_for_batch(decoded_labels, decoded_preds, 10)
    
    # Jaccard@K
    print('Jaccard F1@10...')
    result["jaccard_similarity"] = jaccard_similarity_for_batch(decoded_labels, decoded_preds, 10)
        
    result = {key: round(value * 100, 4) for key, value in result.items()}
    result["gen_len"] = round(np.mean(prediction_lens), 4)
    
    return result

In [ ]:
metrics = compute_metrics((predictions, inputs['labels']))

In [ ]:
metrics

#### Beam Search
```
generation_args = {
    'num_beams': 5, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'early_stopping': False,
}

{'rouge1': 65.9559,
 'rouge2': 45.8405,
 'rougeL': 53.5547,
 'rougeLsum': 53.5547,
 'F1@10': 58.2764,
 'jaccard_similarity': 25.382}
```

#### Diverse Beam Search
```
generation_args = {
    'num_beams': 15, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'num_beam_groups': 3,
    'early_stopping': False,
}

{'rouge1': 65.9841,
 'rouge2': 45.8601,
 'rougeL': 53.5776,
 'rougeLsum': 53.5776,
 'F1@10': 58.2764,
 'jaccard_similarity': 25.382}
```

#### Sampling
```
generation_args = {
    'do_sample': True, 
    'top_k': 3, 
    'top_p': 0.95, 
}

{'rouge1': 64.5375,
 'rouge2': 43.5099,
 'rougeL': 52.8016,
 'rougeLsum': 52.8016,
 'F1@10': 58.6438,
 'jaccard_similarity': 24.1092}
```


# Inference on Eval Dataset

In [ ]:
for data, pred in zip(eval_dataset, predictions):
    pred = np.where(pred != -100, pred, tokenizer.pad_token_id)
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')